# Try a new method

[link to their github repo](https://github.com/ZhengzeZhou/unbiased-feature-importance)

In [1]:
#import shap
import pandas as pd
import scipy as sp
import numpy as np
#from irf.ensemble import wrf
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt
from treeinterpreter.feature_importance import feature_importance as FI
from treeinterpreter.feature_importance import MDA
import sys
sys.path += ['../code/']
from naive import naive_OOB_importance

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [17]:
### load data

num = 2

problem = [0]
tree_depth = [1]
for num in range(1, 3):
    for problem in ['classification', 'regression']:
        for tree_depth in ['deep', 'shallow']:
            #if tree_depth == 'deep':
            #    continue
            #if (num == 1 and tree_depth == 'deep' and problem == 'regression'):
            #    continue
            print('working on {} {} {}'.format(num, problem, tree_depth))
            gini_list, naive_list = [], []
            for i in range(40):
                print(i)
                X_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_train.csv'.format(problem, num, i), delimiter=',')
                y_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_train.csv'.format(problem, num, i), delimiter=',')
                X_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_test.csv'.format(problem, num, i), delimiter=',')
                y_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_test.csv'.format(problem, num, i), delimiter=',')
                noisy_features = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_noisy_features.csv'.format(problem, num, i), delimiter=',').astype(np.int)
                n_features = len(noisy_features)

                ### Train RF
                if problem == 'classification':
                    if tree_depth == 'deep':
                        rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=1)
                    else:
                        rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=100)
                else:
                    if tree_depth == 'deep':
                        rf = rfr(n_estimators=100, max_features= 10, min_samples_leaf=1)
                    else:
                        rf = rfr(n_estimators=100, max_features= 10, min_samples_leaf=100)
                rf.fit(X_train, y_train)
                gini_imp = rf.feature_importances_
                #ufi_imp = UFI_importance(rf, X_train, y_train)
                #ufi_imp[ufi_imp < 0] = 0
                gini_list.append(roc_auc_score(noisy_features, - gini_imp))
                naive_imp = naive_OOB_importance(rf, X_train, y_train)
                #ufi_list.append(roc_auc_score(noisy_features, - ufi_imp))
                naive_list.append(roc_auc_score(noisy_features, -naive_imp))
                
            np.savez('../intermediate/04_aggregate/{}{}/add_new_naive_results_{}.npz'.format(problem, num, tree_depth), gini_list=gini_list, naive_list=naive_list)


working on 1 classification deep
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 1 classification shallow
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 1 regression deep
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 1 regression shallow
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 2 classification deep
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 2 classification shallow
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
working on 2 regression deep
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
wo

In [10]:
problem = 'classification'
tree_depth = 'shallow'
num = 2
i = 10
X_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_train.csv'.format(problem, num, i), delimiter=',')
y_train = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_train.csv'.format(problem, num, i), delimiter=',')
X_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_X_test.csv'.format(problem, num, i), delimiter=',')
y_test = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_y_test.csv'.format(problem, num, i), delimiter=',')
noisy_features = np.loadtxt('../intermediate/04_aggregate/{}{}/permuted{}_noisy_features.csv'.format(problem, num, i), delimiter=',').astype(np.int)

In [11]:
rf = rfc(n_estimators=100, max_features=10, min_samples_leaf=1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [4]:
%timeit naive_imp = naive_OOB_importance(rf, X_train, y_train)

KeyboardInterrupt: 

In [19]:
np.mean(naive_list), np.mean(gini_list)

(0.38472222222222213, 0.39249999999999996)

In [13]:
roc_auc_score(noisy_features, -naive_imp), roc_auc_score(noisy_features, -rf.feature_importances_)

(0.1911111111111111, 0.16)